In [ ]:
#Import required libraries
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt

In [ ]:
#Load dataset into a Pandas DataFrame
df = pd.read_csv('2011-2022_street.csv')

In [ ]:
#Convert month datatype to datetime format
df['Month'] = pd.to_datetime(df['Month'])

In [ ]:
#Rename month column to a more appropriate name
df.rename(columns={'Month':'Year-Month'}, inplace=True)

In [ ]:
#Creating a new column to indicate crime count of 1 for each row
df['Crime_count'] = 1

#Create 3 new dataframes grouped by year, crime type and location against sum of crime incidents
df_yearcount = df.groupby(['Year-Month'])['Crime_count'].sum().reset_index()
df_crimetypecount = df.groupby(['Year-Month', 'Crime type'])['Crime_count'].sum().reset_index()
df_loccount = df.groupby(['Location'])['Crime_count'].sum().reset_index()

In [ ]:
#Code block to plot line chart of number of reported crimes by year
f = plt.figure()
f.set_figwidth(10)
f.set_figheight(6)
plt.plot(df_yearcount['Year-Month'], df_yearcount['Crime_count'])
plt.title('Numbers of Reported Crimes (2011-2022)')
plt.xlabel('Year')
plt.ylabel('Number of Reported Crimes')
plt.show()

In [ ]:
#Generate 2 different columns that states Month and Year name

tqdm.pandas(desc='Month', colour='Green')
df_yearcount['Month'] = df_yearcount.progress_apply(lambda row: row[0].month_name(), axis=1)

tqdm.pandas(desc='Year', colour='Green')
df_yearcount['Year'] = df_yearcount.progress_apply(lambda row: row[0].year, axis=1)

In [ ]:
#Code block to plot number of reported crimes by Calendar Month

import plotly.express as px

fig = px.line(df_yearcount, x='Month', y='Crime_count', color='Year', 
            title='Numbers of Reported Crimes (2011-2022), by Calendar Month')
fig.show()

In [ ]:
#Code block to plot number of reported crimes by Crime Type

fig = px.line(df_crimetypecount, x='Year-Month', y='Crime_count', color='Crime type',
                                labels={'Year', 'Number of Reported Crimes', 'Crime Type'},
                                title='Numbers of Reported Crimes (2011-2022), by Crime Type')
fig.show()

In [ ]:
#Code block to generate treemap of the top 15 crime incident occuring locations

df_loccount = df_loccount.sort_values(by='Crime_count', ascending=False)

fig = px.treemap(df_loccount.head(15), path=['Location'], values='Crime_count', color=df_loccount['Crime_count'].head(15), 
                    hover_data=['Location'], color_continuous_scale='Greens')
fig.show()

In [ ]:
#Import geopandas library and read the .geojson file

import geopandas as gpd

map_df = gpd.read_file('lsoa.geojson')
map_df.head()

In [ ]:
#Rename columns to ensure consistency among all datasets
map_df.rename(columns={'LSOA11CD':'LSOA code'}, inplace=True)
map_df.head()

In [ ]:
#Create a new dataframe to groupby sum of crime incidents in each LSOA code
df_lsoacount = df.groupby(['LSOA code'])['Crime_count'].sum().reset_index()
df_lsoacount.head()

In [ ]:
map_df.plot()

In [ ]:
#Create a new dataframe by merging the Dataframe from the .geojson file and the LSAO crime counts
merged_df = map_df.set_index('LSOA code').join(df_lsoacount.set_index('LSOA code'))
merged_df.head()

In [ ]:
#Code block to plot number of reported crimes for each LSOA in a map

vmin, vmax = 0, 70000
fig, ax = plt.subplots(1, figsize=(15, 6))
merged_df.plot(column='Crime_count', cmap='Reds', linewidth=0.15, ax=ax, edgecolor='0.9')
ax.axis('off')
ax.set_title('Numbers of Reported Crimes (2011-2022), by LSOA', fontdict={'fontsize':'10', 'fontweight':'3'})
sm = plt.cm.ScalarMappable(cmap='Reds', norm=plt.Normalize(vmin=vmin, vmax=vmax))
sm._A = []
cbar = fig.colorbar(sm)

In [ ]:
#Reset Index of the DataFrame to bring back LSOA code column in the DataFrame
merged_df = merged_df.reset_index()
merged_df.head()

In [ ]:
#Derive the centroid coordinates from the DataFrame
x_map=merged_df.centroid.x.mean()
y_map=merged_df.centroid.y.mean()
print(x_map,y_map)

In [ ]:
#Import required map libraries and initialize the map according to the centroids

import folium

mymap = folium.Map(location=[y_map, x_map], zoom_start=11,tiles=None)
folium.TileLayer('CartoDB positron',name="Light Map",control=False).add_to(mymap)
mymap

In [ ]:
#Codeblock to create initial chloropleth map

mymap.choropleth(
 geo_data=merged_df,
 name='Choropleth',
 data=merged_df,
 columns=['LSOA code','Crime_count'],
 key_on="feature.properties.LSOA code",
 fill_color='Reds',
 fill_opacity=1,
 line_opacity=0.2,
 legend_name='Number of Reported Crimes (2011-2022), by LSOA',
 smooth_factor=0
)
mymap

In [ ]:
#Code block to add interactivity and dialogue box with labels inside the map

style_function = lambda x: {'fillColor': '#ffffff', 
                            'color':'#000000', 
                            'fillOpacity': 0.1, 
                            'weight': 0.1}
highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.50, 
                                'weight': 0.1}
LSOA = folium.features.GeoJson(
    merged_df,
    style_function=style_function, 
    control=False,
    highlight_function=highlight_function, 
    tooltip=folium.features.GeoJsonTooltip(
        fields=['LSOA11NM','Crime_count'],
        aliases=['LSOA: ','Total Crime Incidents: '],
        style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;") 
    )
)
mymap.add_child(LSOA)
mymap.keep_in_front(LSOA)
folium.LayerControl().add_to(mymap)
mymap